CS594 Green Line
[Due on Oct19]Step1: Collect data on list of dispensaries from online : 
1. In Phase1, fields are as below:
>name
>street lnie 1,street line 2,city,state,zip code
>tel

2. As for weeds lists, we assume that the dispensaries have their own inventories like a CSV or .xls file containing the usage(recreational, medical, etc), storage(0-10^10 gram), unit prices for each weed items.

3. Pase2:
>image(TBD), open time


In [5]:
import io, time, json
import requests
from bs4 import BeautifulSoup

In [26]:
def parse_page(html):
    """
    Parse the reviews on a single page of a restaurant.
    
    Args:
        html (string): String of HTML corresponding to a Yelp restaurant

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    soup = BeautifulSoup(html,'html.parser')

    urls = soup.find_all('a', itemprop="url")

    #use beautifulsoup CSS selectors to select and construct the review dict
    d_links=[]
    for url in urls:
        d_link = url.get('href')
        d_links.append(d_link)
#     for r in reviews:
#         review_dict = {}
#         r_contents = r.find_all('meta')
#         review_dict['user_id'] = r_contents[0].get('content')
#         review_dict['rating'] = float(r_contents[1].get('content'))
#         review_dict['date'] = r_contents[2].get('content')
#         review_dict['text'] = r.find('p').contents[0]
#         reviews_list.append(review_dict)
    return d_links

In [78]:
def retrieve_html(url, parse_f=parse_page):
    """
    Return the raw HTML at the specified URL.

    Args:
        url (string): 

    Returns:
        status_code (integer):
        raw_html (string): the raw HTML content of the response, properly encoded according to the HTTP headers.
    """
    resp = requests.request('GET', url)
    dispensary_links = parse_f(resp.text)
    #use a list to store all dispensary links
    
    return resp.status_code, dispensary_links

In [79]:
retrieve_html('https://wheresweed.com/illinois/chicago/marijuana-dispensaries')

(200,
 ['https://wheresweed.com/illinois/chicago/marijuana-dispensaries/medmar-chicago',
  'https://wheresweed.com/illinois/rolling-meadows/marijuana-dispensaries/natures-care',
  'https://wheresweed.com/illinois/elmwood-park/marijuana-dispensaries/floramedex',
  'https://wheresweed.com/illinois/chicago/marijuana-dispensaries/green-pinnacle',
  'https://wheresweed.com/illinois/buffalo-grove/marijuana-dispensaries/pdi-medical',
  'https://wheresweed.com/illinois/oak-park/marijuana-dispensaries/seven-point-il',
  'https://wheresweed.com/illinois/chicago/marijuana-dispensaries/chicago-test',
  'https://wheresweed.com/illinois/addison/marijuana-dispensaries/earthmed',
  'https://wheresweed.com/illinois/schaumburg/marijuana-dispensaries/pharmacannis-health-and-wellness-schaumburg',
  'https://wheresweed.com/illinois/naperville/marijuana-dispensaries/3c-dispensary--naperville',
  'https://wheresweed.com/illinois/addison/marijuana-dispensaries/mindful-medical-marijuana-dispensary',
  'https:/

In [59]:
def extract_infos(url):
    """
    Retrieve ALL of the reviews for a single restaurant on Yelp.

    Parameters:
        url (string): Yelp URL corresponding to the restaurant of interest.

    Returns:
        reviews (list): list of dictionaries containing extracted review information
    """
#     1st retrieve the HTML string with specified url
    resp = requests.get(url)
#     resp = requests.request('GET', url)
#     2nd extract the reviews list and url_next
    dispensary_links = parse_page(resp.text)
    total_links = len(dispensary_links)
    dispensaries = []
    
    if  total_links > 0:
        for link in dispensary_links:
            r = requests.get(link)
            dispensary = parse_info(r.text)
            dispensaries.append(dispensary)
    
    return dispensaries

In [75]:
def parse_info(html):
    """
    Parse the reviews on a single page of a restaurant.
    
    Args:
        html (string): String of HTML corresponding to a Yelp restaurant

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    soup = BeautifulSoup(html,'html.parser')
    #<meta property="business:contact_data:street_address"
    store_name = soup.find('h1').contents[0].contents[0]
#     print(store_name.contents)
    street_addr = soup.find('meta', property="business:contact_data:street_address")
    locality = soup.find('meta', property="business:contact_data:locality")
    country = soup.find('meta', property="business:contact_data:country")
    postcode = soup.find('meta', property="business:contact_data:postal_code")
    phone_number = soup.find('meta', property="business:contact_data:phone_number")
    #use beautifulsoup CSS selectors to select and construct the review dict
    store_dict = {}
    store_dict["store_name"] = store_name
    if street_addr != None:
        store_dict["street_addr"] = street_addr.get('content')
    else:
        # give a fake address temporarily
        store_dict["street_addr"] = "807F South Halsted Street"
    if locality != None:
        store_dict["locality"] = locality.get('content')
    else:
        # give a fake one temporarily
        store_dict["locality"] = "Greek Town"
    if country != None:
        store_dict["country"] = country.get('content')
    else:
        store_dict["country"] = "United Streets"
    if postcode != None:
        store_dict["postcode"] = postcode.get('content')
    else:
        store_dict["postcode"] = "60607"
    if phone_number != None:
        store_dict["phone_number"] = phone_number.get('content')
    else:
        store_dict["phone_number"] = "302-8888-0001"
#     for r in reviews:
#         review_dict = {}
#         r_contents = r.find_all('meta')
#         review_dict['user_id'] = r_contents[0].get('content')
#         review_dict['rating'] = float(r_contents[1].get('content'))
#         review_dict['date'] = r_contents[2].get('content')
#         review_dict['text'] = r.find('p').contents[0]
#         reviews_list.append(review_dict)
    return store_dict

In [76]:
# https://potguide.com/illinois/marijuana-dispensaries/chicago/
stores = extract_infos("https://wheresweed.com/illinois/chicago/marijuana-dispensaries")

In [83]:
print(stores, len(stores))
import json

with open('dispensaries.json', 'w') as fp:
    json.dump(stores, fp)

In [81]:
#https://potguide.com/illinois/marijuana-dispensaries/chicago/
def parse_Chicago_dispensaries_page(html):
    """
    Parse the reviews on a single page of a restaurant.
    
    Args:
        html (string): String of HTML corresponding to a Yelp restaurant

    Returns:
        tuple(list, string): a tuple of two elements
            first element: list of dictionaries corresponding to the extracted review information
            second element: URL for the next page of reviews (or None if it is the last page)
    """
    soup = BeautifulSoup(html,'html.parser')

    urls = soup.find_all('div', "basic-listing")

    #use beautifulsoup CSS selectors to select and construct the review dict
    d_links=[]
    for url in urls:
        d_link = url.contents
        print(d_link)
#     for r in reviews:
#         review_dict = {}
#         r_contents = r.find_all('meta')
#         review_dict['user_id'] = r_contents[0].get('content')
#         review_dict['rating'] = float(r_contents[1].get('content'))
#         review_dict['date'] = r_contents[2].get('content')
#         review_dict['text'] = r.find('p').contents[0]
#         reviews_list.append(review_dict)
    return d_links

# stage 2 Some other webiste to collect dispensaries info
TBD

In [82]:
retrieve_html("https://potguide.com/illinois/marijuana-dispensaries/chicago/", parse_Chicago_dispensaries_page)

['\n', <h4><a href="/illinois/marijuana-dispensaries/chicago/columbia-care-chicago/" target="_blank">Columbia Care - Chicago</a></h4>, '\r\n4758 N Milwaukee Ave', <br/>, '\r\n                    Chicago, IL 60630', <br/>, '\r\n                        \r\n                            Phone: (312) 948-9082\r\n                            ', <br/>, '\n', <ul class="list-unstyled list-inline" style="margin-top: 5px;">
<li style="padding: 3px; background: #efefef; border-radius: 4px;"><img alt="Medicinal" data-placement="top" data-toggle="tooltip" src="https://static.potguide.com/images/icon-med.png" style="height: 24px; width: 24px;" title="Medical Sales"/></li>
</ul>, '\n', <a class="btn-u btn-u-xs text-white rounded" href="https://col-care.com/location/chicago?utm_source=PotGuide&amp;sid=159225" style="margin-right: 10px;" target="_blank">Website</a>, '\n', <a class="btn-u btn-u-xs text-white rounded" href="/illinois/marijuana-dispensaries/chicago/columbia-care-chicago/" target="_blank">St

(200, [])